In [1]:
import pandas as pd

from mongo_wrapper.mongo_wrapper import MongoWrapper

import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
MONGO_IP = os.getenv("MONGO_IP")
MONGO_PORT = os.getenv("MONGO_PORT")
MONGO_DB = "news_outlets"
MONGO_USERNAME = os.getenv("MONGO_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")

In [6]:
mongo_client = MongoWrapper(
    ip=MONGO_IP,
    port=MONGO_PORT,
    db=MONGO_DB,
    user=MONGO_USERNAME,
    password=MONGO_PASSWORD,
)

2025-05-16 23:21:21,528 WARNING Logger Mongo was configured with True console stream


2025-05-16 23:21:21,854 INFO Connected to news_outlets database on 206.81.16.39


In [7]:
mongo_client.get_all_collections()

['mediacloud_Jalisco',
 'mediacloud_Tamaulipas',
 'mediacloud_Hidalgo',
 'mediacloud_Baja California Sur',
 'mediacloud_Tabasco',
 'mediacloud_Nayarit',
 'mediacloud_Guanajuato',
 'mediacloud_Colima',
 'mediacloud_Veracruz',
 'gnews_desempleo',
 'mediacloud_Sinaloa',
 'mediacloud_Durango',
 'mediacloud_Sonora',
 'mediacloud_Quintana Roo',
 'mediacloud_Puebla',
 'mediacloud_Baja California',
 'mediacloud_Nuevo Leon',
 'mediacloud_desempleo',
 'mediacloud_Campeche',
 'mediacloud_Queretaro',
 'mediacloud_Tlaxcala',
 'mediacloud_San Luis Potosi',
 'mediacloud_Coahuila',
 'mediacloud_Aguascalientes',
 'mediacloud_Oaxaca',
 'mediacloud_Ciudad de México',
 'mediacloud_Chiapas',
 'mediacloud_Chihuahua',
 'mediacloud_Michoacan',
 'mediacloud_Mexico',
 'mediacloud_Zacatecas',
 'mediacloud_Yucatan',
 'mediacloud_Guerrero',
 'mediacloud_Morelos']

In [9]:
result = mongo_client.get_collection_entries(collection="mediacloud_Chihuahua")
result = pd.DataFrame(result)
result

,_id,language,media_name,media_url,publish_date,title,url,content
0,048a12f11b877d29754fa3675794ba15e67a91e0583417...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-01-09,Marchan 120 ciudadanos en contra del reemplaca...,https://noticiasdequeretaro.com.mx/2022/01/09/...,Videos\nMarchan 120 ciudadanos en contra del r...
1,c1e18da40b3ea2815d3f45a5ae58ab6c99eb0678c30920...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-11-02,Lisa se convierte a huracán categoría 1,https://noticiasdequeretaro.com.mx/2022/11/02/...,Nacional\nLisa se convierte a huracán categorí...
2,8c927bf19b6e8438d75a8975e3c7ad439ea35b98c8588e...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2023-10-27,El Frente Frío 8 traerá las primeras nevadas a...,https://noticiasdequeretaro.com.mx/2023/10/27/...,Nacional\nEl Frente Frío 8 traerá las primeras...
3,e24a99df531e0e097f4bca4005af14105b4c0c3d32f817...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-12-21,La Navidad llega con una onda gélida y un fren...,https://noticiasdequeretaro.com.mx/2022/12/21/...,Portada\nLa Navidad llega con una onda gélida ...
4,c916e548af4bef8220e7a0d80e54e63fbcd953cc77984c...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2023-10-11,Lidia se convierte en Huracán,https://noticiasdequeretaro.com.mx/2023/10/11/...,Nacional\nLidia se convierte en Huracán\n-\nPi...
...,...,...,...,...,...,...,...,...
1995,c1f42f13fc50d10dfea7f92fedc329a2f33a95b84260f2...,es,diariodexalapa.com.mx,diariodexalapa.com.mx,2022-01-29,Colectivos exigen justicia para la activista A...,https://www.diariodexalapa.com.mx/republica/so...,NaN
1996,ab37784dee8d22c3414a69d1f4d437292b3997338cf457...,es,elsoldehidalgo.com.mx,elsoldehidalgo.com.mx,2022-01-29,Colectivos exigen justicia para la activista A...,https://www.elsoldehidalgo.com.mx/republica/so...,NaN
1997,8820b385630e5677d9ac44a8fa3fdbba221d22b2f4c002...,es,elheraldodechiapas.com.mx,elheraldodechiapas.com.mx,2022-01-29,Colectivos exigen justicia para la activista A...,https://www.elheraldodechiapas.com.mx/republic...,NaN
1998,6b314301a40c0107b8d742e0c505d34a8eb716f8e8fa63...,es,elheraldodechihuahua.com.mx,elheraldodechihuahua.com.mx,2022-01-29,Colectivos exigen justicia para la activista A...,https://www.elheraldodechihuahua.com.mx/republ...,NaN


In [17]:
states = [
        "Chiapas", "Chihuahua", "Coahuila", "Colima", "Durango", "Guerrero",
        "Hidalgo", "Jalisco", "Mexico", "Michoacan", "Morelos", "Nayarit",
        "Nuevo Leon", "Oaxaca", "Puebla", "Queretaro", "Quintana Roo",
        "San Luis Potosi", "Sinaloa", "Sonora", "Tabasco", "Tamaulipas",
        "Tlaxcala", "Veracruz", "Yucatan", "Ciudad de México", "Zacatecas"
    ]

In [18]:
data = pd.DataFrame()
for state in states:
    # Get the collection for each state
    gnews_collection = mongo_client.get_collection_entries(collection=f"mediacloud_{state}")

    state_collection = pd.DataFrame(gnews_collection)

    data = pd.concat([data, state_collection], ignore_index=True)

In [19]:
data.head()

,_id,language,media_name,media_url,publish_date,title,url,content
0,c1e18da40b3ea2815d3f45a5ae58ab6c99eb0678c30920...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-11-02,Lisa se convierte a huracán categoría 1,https://noticiasdequeretaro.com.mx/2022/11/02/...,Nacional\nLisa se convierte a huracán categorí...
1,8c927bf19b6e8438d75a8975e3c7ad439ea35b98c8588e...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2023-10-27,El Frente Frío 8 traerá las primeras nevadas a...,https://noticiasdequeretaro.com.mx/2023/10/27/...,Nacional\nEl Frente Frío 8 traerá las primeras...
2,5c5c05ec71650a0dc5ea6d4226fadc629403b720f552d5...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-08-01,Cubanas y venezolanas viven situación de riesg...,https://noticiasdequeretaro.com.mx/2022/08/01/...,Alertan sobre redes de trata de personas en el...
3,a73c7786a653cc36ce6d707533cdde74c18c9ca5026c48...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-10-27,El matrimonio igualitario ya es legal en todo ...,https://noticiasdequeretaro.com.mx/2022/10/27/...,NaN
4,fd1b7bb57d10f0c6bc6793e108c8c8d65a77459bdafa1a...,es,noticiasdequeretaro.com.mx,noticiasdequeretaro.com.mx,2022-09-07,Sigue fortaleciéndose “Kay”; Se convierte en h...,https://noticiasdequeretaro.com.mx/2022/09/07/...,NaN


In [23]:
non_nan_content_rows = data['content'].notna().sum()
non_nan_content_rows

np.int64(39324)